### 1) Build a multi-class classification model using CNN

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential, model_from_json
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report

Using TensorFlow backend.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2, # random application of shearing
    zoom_range = 0.2,
    horizontal_flip = True) # randomly flipping half of the images horizontally

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
batch_size = 32
num_val_samples = 137
steps = 2295*1.4/batch_size

In [ ]:
train_generator = train_datagen.flow_from_directory(
    'Jul_2020/train',
    target_size = (256, 256),
    color_mode = 'rgb',
    batch_size = batch_size,
    class_mode = 'sparse')

Found 35448 images belonging to 4 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(
    'Jul_2020/test',
    target_size = (256, 256),
    color_mode = 'rgb',
    batch_size = 1,
    class_mode = 'sparse',
    shuffle = False)

Found 8862 images belonging to 4 classes.


In [ ]:
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(4, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_generator, validation_data = test_generator, epochs=5)

Epoch 1/5
1108/1108 [==============================] - 513s 463ms/step - loss: 0.3691 - accuracy: 0.8834 - val_loss: 0.1923 - val_accuracy: 0.9353
Epoch 2/5
1108/1108 [==============================] - 493s 445ms/step - loss: 0.1736 - accuracy: 0.9417 - val_loss: 0.1304 - val_accuracy: 0.9568
Epoch 3/5
1108/1108 [==============================] - 490s 442ms/step - loss: 0.2198 - accuracy: 0.9237 - val_loss: 0.2561 - val_accuracy: 0.9107
Epoch 4/5
1108/1108 [==============================] - 486s 439ms/step - loss: 0.1776 - accuracy: 0.9420 - val_loss: 0.1111 - val_accuracy: 0.9632
Epoch 5/5
1108/1108 [==============================] - 483s 436ms/step - loss: 0.1146 - accuracy: 0.9627 - val_loss: 0.0914 - val_accuracy: 0.9729


In [ ]:
model.save_weights('drive/My Drive/AML/Model1_9627.h5')

In [ ]:
model.load_weights('drive/My Drive/AML/Model1_9627.h5')

In [ ]:
Y_pred = model.predict(test_generator)

In [ ]:
Y_pred

array([[9.9997556e-01, 2.1733847e-07, 2.4185747e-05, 5.4570840e-12],
       [9.6222925e-01, 3.6400910e-02, 1.4664066e-05, 1.3551178e-03],
       [9.6244675e-01, 3.7369747e-02, 1.8315675e-04, 3.5429127e-07],
       ...,
       [1.8551290e-02, 2.6097070e-05, 1.7067454e-04, 9.8125196e-01],
       [8.6448668e-03, 7.4338786e-05, 1.0301232e-04, 9.9117774e-01],
       [1.0174648e-03, 3.6953195e-05, 1.2113347e-07, 9.9894553e-01]],
      dtype=float32)

In [ ]:
y_pred = np.argmax(Y_pred, axis=1)
y_pred

array([0, 0, 0, ..., 3, 3, 3])

##### Confusion Matrix

In [ ]:
print(confusion_matrix(test_generator.classes, y_pred))

[[2176   63   10    9]
 [  40 4229    2    8]
 [  16    3 1825    0]
 [  59   26    4  392]]


In [ ]:
target_names = ['Accessories', 'Apparel', 'Footware', 'Personal Care']

In [ ]:
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

               precision    recall  f1-score   support

  Accessories       0.95      0.96      0.96      2258
      Apparel       0.98      0.99      0.98      4279
     Footware       0.99      0.99      0.99      1844
Personal Care       0.96      0.81      0.88       481

     accuracy                           0.97      8862
    macro avg       0.97      0.94      0.95      8862
 weighted avg       0.97      0.97      0.97      8862



### 2) Build a model with 2 branches: 1st branch is a CNN, 2nd branch is a pure fully connected network

In [ ]:
model1 = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu')
])

In [ ]:
model2 = tf.keras.Sequential([
         layers.Flatten(input_shape=(256, 256, 3)),
         layers.Dense(64, activation='relu'),
         layers.Dropout(0.5),
         layers.Dropout(0.5)                     
])

In [ ]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 516128)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                33032256  
Total params: 33,033,152
Trainable params: 33,033,152
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model2.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 196608)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                12582976  
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
Total params: 12,582,976
Trainable params: 12,582,976
Non-trainable params: 0
_________________________________________________________________


In [ ]:
common_input = layers.Input((256, 256, 3))
out1 = model1(common_input)
out2 = model2(common_input)

In [ ]:
merged = tf.keras.layers.Concatenate()([out1, out2])

In [ ]:
#mergedOut = tf.keras.layers.Flatten()(merged)

In [ ]:
result = tf.keras.layers.Dense(64, activation='softmax')(merged)

In [ ]:
from tensorflow.keras.models import Model

In [ ]:
Model3 = Model(common_input, result)

In [ ]:
Model3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
Model3.fit(train_generator, validation_data = test_generator, epochs=5)

Epoch 1/5
1108/1108 [==============================] - 614s 554ms/step - loss: 1.0761 - accuracy: 0.8540 - val_loss: 0.2408 - val_accuracy: 0.9211
Epoch 2/5
1108/1108 [==============================] - 610s 550ms/step - loss: 0.3357 - accuracy: 0.9023 - val_loss: 0.2349 - val_accuracy: 0.9245
Epoch 3/5
1108/1108 [==============================] - 607s 547ms/step - loss: 0.2477 - accuracy: 0.9188 - val_loss: 0.2172 - val_accuracy: 0.9300
Epoch 4/5
1108/1108 [==============================] - 608s 548ms/step - loss: 0.2918 - accuracy: 0.9174 - val_loss: 0.2098 - val_accuracy: 0.9332
Epoch 5/5
1108/1108 [==============================] - 608s 549ms/step - loss: 0.1997 - accuracy: 0.9340 - val_loss: 0.1745 - val_accuracy: 0.9403


In [ ]:
Model3.save_weights('drive/My Drive/AML/Model3_9340.h5')

In [ ]:
Model3.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv2d_3_input (InputLayer)     [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
flatten_5_input (InputLayer)    [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 254, 254, 32) 896         conv2d_3_input[0][0]             
__________________________________________________________________________________________________
flatten_5 (Flatten)             (None, 196608)       0           flatten_5_input[0][0]            
____________________________________________________________________________________________